#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces

Ultilizar api do bling para cadastro de itens   (quando tiver)


Mercado LIVRE:50% do valor do produto (para produtos até R$ 12,50), R$ 6,25 (entre R$ 12,50 e R$ 29), R$ 6,50 (entre R$ 29 e R$ 50) e R$ 6,75 (entre R$ 50 e R$ 79)

Comissão de Categoria Construção 12%


SHOPE taxa de transação é sobre o total do produto então é 2% de taxa e 18 de comissão


In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


In [ ]:
#Baixa os anexos XML dos emails

save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [ ]:
# Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    data_str = root.find(".//ns:ide/ns:dhEmi", ns).text
    data_emissao = datetime.fromisoformat(data_str)

    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        temp_codigo = det.find("./ns:prod/ns:cProd", ns).text
        if not "-" in temp_codigo:
            produto ['Codigo Produto'] = temp_codigo
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produto ['Data Emissão'] = data_emissao

        produtos.append(produto)
    return produtos


# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns).text
            
            pasta_destino = os.path.join("notas/")

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")



fornecedores_pesados = [
    'IND E COM DE TUBOS E CONEXOES FORT.COM',
    'VOTORANTIM CIMENTOS SA',
    'CABOQUINHO MATERIAIS PARA CONSTRUCAO'
]


produtos = pd.DataFrame(todos_produtos)
produtos = produtos[~produtos['Fornecedor'].isin(fornecedores_pesados)]
produtos = produtos.sort_values(by="Data Emissão", ascending=False)

produtos = produtos.drop_duplicates(subset='Codigo Produto', keep='first')



In [ ]:
produtos['Fornecedor'].unique()

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
}



In [ ]:
for  index, produto in produtos.iterrows():
    fornecedor = produto['Fornecedor']
    codigo_produto = produto['Codigo Produto']
    descricao = produto['Descrição']
    try:
        if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':   
            url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
            url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
            url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}'
        else:
            print('Indisponivel')
            continue
        
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        script = soup.find("script", type="application/json")
        data = json.loads(script.string) if script else {}

        extrai = data.get("props", {}).get("pageProps", {}).get("produto", {})
        url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
        marca = next((p.get("desc") for p in extrai.get("dimensoes", []) if p.get("label") == "MARCA"), "Não disponível")
        peso = extrai.get("pesoBruto", "Não disponível")
        if codigo_barras == 'SEM GTIN':
            codigo_barras = extrai.get("codBarra", "SEM GTIN")
        produtos.at[index, 'Código de Barras'] = codigo_barras
        produtos.at[index, 'Peso'] = peso
        produtos.at[index, "Marca"] = marca
        produtos.at[index, 'Url Imagem'] = url_img
        
        print(produto['Descrição'])
        
    except Exception as e:
        print(e)


produtos.to_excel('produtos.xlsx', index=False)


In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz

# === Configurações ===
threshold = 75  # similaridade mínima
arquivo_produtos = "produtos.xlsx"
arquivo_saida_fuzzy = "Fuzzy.xlsx"
arquivo_saida_bling = "produtos_para_bling.xlsx"

# === Leitura inicial ===
df = pd.read_excel(arquivo_produtos)

# === Agrupamento Fuzzy ===
grupos, grupo_id, grupo_coluna = {}, 1, []

for desc in df["Descrição"]:
    # Se já está em algum grupo, apenas reutiliza o ID
    gid_existente = next((gid for gid, itens in grupos.items() if desc in itens), None)
    if gid_existente:
        grupo_coluna.append(gid_existente)
        continue

    # Cria novo grupo
    grupos[grupo_id] = [desc]
    grupo_coluna.append(grupo_id)

    # Busca similares
    for match_desc, score, _ in process.extract(desc, df["Descrição"], scorer=fuzz.token_sort_ratio, limit=None):
        if score >= threshold and match_desc != desc:
            grupos[grupo_id].append(match_desc)

    grupo_id += 1

df["Grupo"] = grupo_coluna

# Filtra linhas com peso válido
df = df[df["peso"].astype(str).str.upper() != "NÃO DISPONIVEL"]

# Padroniza colunas
df.columns = df.columns.str.strip().str.lower()

# Garante nome da coluna de código
if "codigo" not in df.columns:
    for c in df.columns:
        if "código" in c or "codigo" in c:
            df.rename(columns={c: "codigo"}, inplace=True)
            break

# Cria estrutura de hierarquia
bling_data = [
    {"codigo": row["codigo produto"], "codigo pai": "" if idx == 0 else grupo.iloc[0]["codigo produto"]}
    for _, grupo in df.groupby("grupo") 
    for idx, row in grupo.reset_index(drop=True).iterrows()
]

bling_df = pd.DataFrame(bling_data)

# Junta tudo
bling_final = df.merge(bling_df, on="codigo", how="right")
bling_final.to_excel(arquivo_saida_bling, index=False)

print("✅ Planilha gerada com sucesso:", arquivo_saida_bling)


In [12]:
import pandas as pd

# Carregar a planilha
df = pd.read_excel("produtos_para_bling.xlsx")

# Padronizar nomes das colunas
df.columns = df.columns.str.strip().str.lower()

# Remover linhas onde 'peso' OU 'url imagem' tenham o valor "NÃO DISPONIVEL"
df_filtrado = df[
    (df["peso"].astype(str).str.upper() != "NÃO DISPONIVEL") &
    (df["url imagem"].astype(str).str.upper() != "NÃO DISPONIVEL")
]

# Salvar resultado
df_filtrado.to_excel("Fuzzy_filtrado.xlsx", index=False)

df_filtrado
print(df_filtrado.to_string(index=False))


    codigo                                                            descrição  valor_unitário código de barras                                      sku                                                  fornecedor        data emissão        ncm               marca      peso                                                url imagem  codigo pai
  130416.0                                    SUP LAJE PE FIXO 30CM AM INJEREST        3.540000    7898759180523                            7898759180523                    CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU 2025-10-03 05:27:00 39174090.0            INJEREST  0.135000   https://cdn.construja.com.br/produtos/130416/130416.jpg         NaN
  130415.0                                    SUP LAJE PE FIXO 30CM PT INJEREST        2.501000    7898759180516                            7898759180516                    CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU 2025-10-03 05:27:00 39174090.0            INJEREST  0.122000   https://cdn.construja.com.br/prod

In [5]:
shopee_fuzzy = pd.read_excel('Fuzzy.xlsx') 


# Padronizar nomes das colunas
shopee_fuzzy.columns = shopee_fuzzy.columns.str.strip().str.lower()

# Remover linhas onde 'peso' OU 'url imagem' tenham o valor "NÃO DISPONIVEL"
shopee_fuzzy_filtrado = shopee_fuzzy[
    (shopee_fuzzy["peso"].astype(str).str.upper() != "NÃO DISPONIVEL") &
    (shopee_fuzzy["url imagem"].astype(str).str.upper() != "NÃO DISPONIVEL")
]

shopee_fuzzy_filtrado.keys()


Index(['codigo produto', 'descrição', 'valor_unitário', 'código de barras',
       'sku', 'fornecedor', 'data emissão', 'ncm', 'marca', 'peso',
       'url imagem', 'grupo'],
      dtype='object')

In [ ]:


import pandas as pd
from rapidfuzz import fuzz, process

# === 1. Ler o arquivo Excel ===
df = pd.read_excel("todos_produtos.xlsx")


# # Verifica se a coluna existe
# if "Descrição" not in df.columns:
#     raise ValueError("A planilha deve conter uma coluna chamada 'Descrição'")

# # === 2. Preparar os dados ===
# df['Descrição'] = df['Descrição'].astype(str).str.strip()
# produtos = df['Descrição'].tolist()

# # === 3. Agrupar produtos semelhantes (usando similaridade fuzzy) ===
# grupos = []
# visitados = set()
# grupo_id = 0
# similaridade_minima = 85  # ajuste se quiser mais ou menos rigoroso

# for i, prod in enumerate(produtos):
#     if i in visitados:
#         continue

#     grupo_id += 1
#     grupo = [prod]
#     visitados.add(i)

#     for j, outro in enumerate(produtos):
#         if j in visitados:
#             continue
#         if fuzz.token_sort_ratio(prod, outro) >= similaridade_minima:
#             grupo.append(outro)
#             visitados.add(j)

#     grupos.append((grupo_id, grupo))

# # === 4. Extrair base e variação de cada grupo ===
# linhas = []

# for gid, grupo in grupos:
#     if len(grupo) == 1:
#         linhas.append({
#             "Descrição": grupo[0],
#             "Base": grupo[0],
#             "Variação": "",
#             "Grupo": gid
#         })
#         continue

#     # Pega o primeiro como referência
#     ref = grupo[0].split()

#     # Palavras comuns entre todos os produtos do grupo
#     comuns = set(ref)
#     for g in grupo[1:]:
#         comuns &= set(g.split())

#     # Cria as linhas
#     for g in grupo:
#         palavras = g.split()
#         base = " ".join([p for p in palavras if p in comuns])
#         variacao = " ".join([p for p in palavras if p not in comuns])
#         linhas.append({
#             "Descrição": g,
#             "Base": base.strip(),
#             "Variação": variacao.strip(),
#             "Grupo": gid
#         })

# # === 5. Salvar resultado ===
# resultado = pd.DataFrame(linhas)
# resultado.to_excel("produtos_agrupados.xlsx", index=False)

# print("✅ Arquivo salvo como 'produtos_agrupados.xlsx'")


,ID,Código,Descrição,Unidade,NCM,Origem,Preço,Valor IPI fixo,Observações,Situação,...,Número FCI,Vídeo,Departamento,Unidade de medida,Preço de compra,Valor base ICMS ST para retenção,Valor ICMS ST para retenção,Valor ICMS próprio do substituto,Categoria do produto,Informações Adicionais
30,NaN,194319.0,SIFAO FLEX ANEL PVC PT ASTRA,UN,39174090.0,0,5.130,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,5.130,NaN,NaN,NaN,NaN,NaN
31,NaN,194141.0,SIFAO CR FLEX ANEL PVC ASTRA,UN,39174090.0,0,23.207,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,23.207,NaN,NaN,NaN,NaN,NaN
32,NaN,18281.0,SIFAO BR FLEX ANEL PVC,UN,39174090.0,0,3.577,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,3.577,NaN,NaN,NaN,NaN,NaN
33,NaN,13651.0,COLA EPOXI BICOMP MASSA 50G DUREPOXI HENKEL,UN,35061090.0,0,4.950,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,4.950,NaN,NaN,NaN,NaN,NaN
34,NaN,13652.0,COLA EPOXI BICOMP MASSA 100G DUREPOXI HENKEL,UN,35061090.0,0,7.490,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,7.490,NaN,NaN,NaN,NaN,NaN
35,NaN,13702.0,REGULADOR GAS 504/1 MED ALIANÇA,UN,84811000.0,0,25.440,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,25.440,NaN,NaN,NaN,NaN,NaN
36,NaN,6269.0,REGULADOR GAS MED 504/01 - ALIANCA ALIANÇA,UN,84811000.0,0,27.363,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,27.363,NaN,NaN,NaN,NaN,NaN
37,NaN,6268.0,REGULADOR GAS GRD 506/01 - ALIANCA ALIANÇA,UN,84811000.0,0,38.591,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,38.591,NaN,NaN,NaN,NaN,NaN
38,NaN,53201.0,REGULADOR GAS ALIANCA GRANDE 506/01,UN,84811000.0,0,34.190,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,34.190,NaN,NaN,NaN,NaN,NaN
39,NaN,216291.0,REGULADOR GAS ALIANCA MEDIO 504/01,UN,84811000.0,0,25.590,NaN,NaN,Ativo,...,NaN,NaN,NaN,Centímetro,25.590,NaN,NaN,NaN,NaN,NaN
